In [1]:
from tensorflow.keras.layers import LSTM

class CustomLSTM(LSTM):
    def __init__(self, *args, **kwargs):
        kwargs.pop('time_major', None)  # Remove the 'time_major' argument if it exists
        super(CustomLSTM, self).__init__(*args, **kwargs)

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({'CustomLSTM': CustomLSTM})

trained_model = load_model(r"C:\Users\pranali\Desktop\Throughput_prediction\4G-Models\Models\univariate_classifier.hdf5", custom_objects={'LSTM': CustomLSTM})

trained_model.summary()

C:\Users\pranali\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\pranali\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (CustomLSTM)               │ (None, 10, 160)        │       103,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 160)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (CustomLSTM)             │ (None, 10, 128)        │       147,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (CustomLSTM)             │ (None, 10, 80)         │        66,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 10, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 48)             │        38,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 56)             │         2,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 56)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 48)             │         2,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ OUT_univariate_classifier       │ (None, 3)              │           147 │
│ (Dense)                         │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,605 (1.38 MB)

 Trainable params: 362,603 (1.38 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
config = trained_model.get_config() # Returns pretty much every information about your model
print(config["layers"][0]) # returns a tuple of width, height and channels

{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': (None, 10, 1), 'dtype': 'float32', 'sparse': False, 'name': 'lstm_input'}, 'registered_name': None}


In [5]:
#4g

import numpy as np
import tensorflow as tf


X_train = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\test_train_vis\Training\univariate_classifier_train_x.npy")  # Shape should be (num_samples, time_steps, num_features)
y_train = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\test_train_vis\Training\univariate_classifier_train_y.npy")  # Shape should be (num_samples, num_classes)
X_test = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\test_train_vis\Testing\univariate_classifier_test_x.npy")
y_test = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\test_train_vis\Testing\univariate_classifier_test_y.npy")

# Ensure reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [9]:
print(f"X_train : {X_train.shape}")
print(f"y_train : {y_train.shape}")
print(f"X_test : {X_test.shape}")
print(f"y_test : {y_test.shape}")

X_train : (131468, 10, 1)
y_train : (131468, 3)
X_test : (26643, 10, 1)
y_test : (26643, 3)


In [18]:
#comparing model size

import os

def get_model_size(model):
    try:
        model_size = sum([np.prod(var.shape) for var in model.trainable_variables]) * 4
        return model_size / (1024 * 1024)  # MB
    except Exception as e:
        print(f"Error calculating model size: {e}")
        return None

# Model size in MB
model_size_params = trained_model.count_params()
model_size_bytes = model_size_params * 4  # 4 bytes per float32 parameter
model_size_mb = model_size_bytes / (1024 ** 2)  # Convert bytes to megabytes
print(f"Original Model Size: {model_size_mb:.2f} MB")
model_size_kb = model_size_bytes / 1024  # Convert bytes to kilobytes
print(f"Original Model Size: {model_size_kb:.2f} KB")


Original Model Size: 1.38 MB
Original Model Size: 1416.42 KB


In [36]:
 #Evaluate the model on test data
loss, accuracy = trained_model.evaluate(X_test, y_test, verbose=1)

# Print the evaluation results
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

833/833 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.4996 - loss: 10.5203
Test Loss: 9.082098960876465
Test Accuracy: 0.5688173174858093


In [11]:
import time
import numpy as np

# Assuming CustomLSTM and trained_model are already defined
# Ensure X_test is your test dataset

# Start timing
start_time = time.time()

# Loop through each sample in the test dataset
for i in range(len(X_test)):
    # Predict the class probabilities for the sample
    y_pred = trained_model.predict(np.expand_dims(X_test[i], axis=0), verbose=0)
    
    # Convert probabilities to class prediction (use argmax for classification)
    predicted_class = np.argmax(y_pred, axis=-1)
    
    # Print the index (optional, can be commented out if unnecessary)
    print(f"Sample {i}: Predicted class {predicted_class}")

# End timing
end_time = time.time()

# Calculate total inference time
inference_time_4g = (end_time - start_time)
print(f"Inference Time for all 4G test samples: {inference_time_4g:.6f} seconds")

# Calculate average inference time per sample
avg_inference_time_4g = inference_time_4g / len(X_test)
print(f"Avg Inference Time per sample for 4G test data: {avg_inference_time_4g:.6f} seconds")


Sample 0: Predicted class [2]
Sample 1: Predicted class [2]
Sample 2: Predicted class [2]
Sample 3: Predicted class [2]
Sample 4: Predicted class [2]
Sample 5: Predicted class [2]
Sample 6: Predicted class [2]
Sample 7: Predicted class [2]
Sample 8: Predicted class [2]
Sample 9: Predicted class [2]
Sample 10: Predicted class [2]
Sample 11: Predicted class [2]
Sample 12: Predicted class [2]
Sample 13: Predicted class [2]
Sample 14: Predicted class [2]
Sample 15: Predicted class [2]
Sample 16: Predicted class [2]
Sample 17: Predicted class [2]
Sample 18: Predicted class [2]
Sample 19: Predicted class [2]
Sample 20: Predicted class [2]
Sample 21: Predicted class [2]
Sample 22: Predicted class [2]
Sample 23: Predicted class [2]
Sample 24: Predicted class [2]
Sample 25: Predicted class [2]
Sample 26: Predicted class [2]
Sample 27: Predicted class [2]
Sample 28: Predicted class [2]
Sample 29: Predicted class [2]
Sample 30: Predicted class [2]
Sample 31: Predicted class [2]
Sample 32: Predict

In [13]:
print(f"Shape of y_true: {y_test.shape}")
print(f"Shape of y_pred: {y_pred.shape}")


Shape of y_true: (26643, 3)
Shape of y_pred: (1, 3)


In [60]:
import time
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Assuming X_test and y_test are your test dataset and true labels
# CustomLSTM and trained_model are already defined

# Initialize lists to hold true and predicted labels
y_true = []
y_pred = []

# Initialize variables for timing
start_time = time.time()

# Loop through each sample in the test dataset and predict the class
for i in range(len(X_test)):
    # Start time for this sample
    sample_start_time = time.time()
    
    # Predict the class probabilities for the sample
    y_prob = trained_model.predict(np.expand_dims(X_test[i], axis=0), verbose=0)
    
    # Convert probabilities to predicted class label
    predicted_class = np.argmax(y_prob, axis=-1)
    
    # Append the predicted class and true class to lists
    y_pred.append(predicted_class[0])  # [0] to extract scalar from array
    
    # If y_test is one-hot encoded, convert it to class labels
    true_class = np.argmax(y_test[i], axis=-1)  # Ensure this matches the predicted format
    y_true.append(true_class)
    
    # End time for this sample (optional to print per-sample inference time)
    sample_end_time = time.time()
    sample_inference_time = sample_end_time - sample_start_time
    print(f"Sample {i}: Inference Time = {sample_inference_time:.6f} seconds")

# End total timing
end_time = time.time()

# Calculate total and average inference time
total_inference_time = end_time - start_time
avg_inference_time = total_inference_time / len(X_test)

# Print the inference time
print(f"Total Inference Time for all samples: {total_inference_time:.6f} seconds")
print(f"Average Inference Time per sample: {avg_inference_time:.6f} seconds")

# Convert lists to numpy arrays for scikit-learn compatibility
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Print the confusion matrix
labels = [0, 1, 2]  # Assuming 0: low, 1: medium, 2: high
cm = confusion_matrix(y_true, y_pred, labels=labels)
print("\nConfusion Matrix:")
print(cm)

# Print the classification report
target_names = ['low', 'medium', 'high']
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=target_names))


Sample 0: Inference Time = 0.105135 seconds
Sample 1: Inference Time = 0.091092 seconds
Sample 2: Inference Time = 0.079163 seconds
Sample 3: Inference Time = 0.067834 seconds
Sample 4: Inference Time = 0.065728 seconds
Sample 5: Inference Time = 0.064231 seconds
Sample 6: Inference Time = 0.067382 seconds
Sample 7: Inference Time = 0.072517 seconds
Sample 8: Inference Time = 0.069884 seconds
Sample 9: Inference Time = 0.067430 seconds
Sample 10: Inference Time = 0.064951 seconds
Sample 11: Inference Time = 0.069674 seconds
Sample 12: Inference Time = 0.069088 seconds
Sample 13: Inference Time = 0.071385 seconds
Sample 14: Inference Time = 0.085655 seconds
Sample 15: Inference Time = 0.072230 seconds
Sample 16: Inference Time = 0.072817 seconds
Sample 17: Inference Time = 0.070340 seconds
Sample 18: Inference Time = 0.077488 seconds
Sample 19: Inference Time = 0.069020 seconds
Sample 20: Inference Time = 0.078500 seconds
Sample 21: Inference Time = 0.074707 seconds
Sample 22: Inference

C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
